In [100]:
import os  
import json
import pandas as pd
import traceback


In [101]:
from langchain.chat_models import ChatOpenAI


In [102]:

from langchain.llms import OpenAI

In [103]:
llm = ChatOpenAI(
    openai_api_key="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",
    base_url="https://openrouter.ai/api/v1",
    model="openai/gpt-3.5-turbo",
)

In [104]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000224924AF090>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000224923E9490>, model_name='openai/gpt-3.5-turbo', model_kwargs={}, openai_api_key='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', openai_api_base='https://openrouter.ai/api/v1', openai_proxy='')

In [105]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [106]:
RESPONSE_JSON = {

    "1": {

        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },

    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },

    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [107]:
TEMPLATE="""
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple + questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQS
### RESPONSE_JSON
{response_json}

"""

In [108]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
    )

In [109]:
quiz_chain = LLMChain(llm=llm , prompt=quiz_generation_prompt, output_key = "quiz", verbose= True)

In [110]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\

You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""


In [111]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject","quiz"], template=TEMPLATE)

In [112]:

review_chain = LLMChain(
    llm=llm,
    prompt=quiz_evaluation_prompt,
    output_key="review",
    verbose=True
)




In [113]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)


In [114]:
file_path=r"C:\Users\modgi\mcqgen\data.txt"

In [115]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [116]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[4][5]

From a theoretical viewpoint, probably approximately c

In [117]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [119]:
NUMBER = 5  # Number of MCQs you want
SUBJECT = "ML"
TONE = "Simple and clear"


with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE, 
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )    

C:\Users\modgi\AppData\Local\Temp\ipykernel_26172\2380114245.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory

In [120]:
print(f"{cb.total_tokens}")
print(f"{cb.prompt_tokens}")
print(f"{cb.completion_tokens}")
print(f"{cb.total_cost}")

1609
862
747
0.0


In [121]:
response 


{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[4][5]\n\nFrom a theoretical viewpoint, probably

In [134]:
quiz = response.get("quiz")
quiz=json.loads(quiz)

In [136]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    option = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq, "choice":option, "Correct":correct})

In [137]:
quiz_table_data

[{'MCQ': 'What is machine learning?',
  'choice': 'a: A field of study in artificial intelligence concerned with developing and studying statistical algorithms | b: A field of study in artificial intelligence only concerned with data analysis | c: A field of study in artificial intelligence focused on robotics | d: A field of study in artificial intelligence focused on computer programming',
  'Correct': 'a'},
 {'MCQ': 'What is the application of machine learning to business problems known as?',
  'choice': 'a: Predictive analytics | b: Natural language processing | c: Computer vision | d: Data mining',
  'Correct': 'a'},
 {'MCQ': 'Which field of study focuses on exploratory data analysis via unsupervised learning?',
  'choice': 'a: Machine learning | b: Data mining | c: Deep learning | d: Artificial intelligence',
  'Correct': 'b'},
 {'MCQ': 'What class of statistical algorithms have surpassed many previous machine learning approaches in performance?',
  'choice': 'a: Neural networks 

In [141]:
pd.DataFrame(quiz_table_data)

,MCQ,choice,Correct
0,What is machine learning?,a: A field of study in artificial intelligence...,a
1,What is the application of machine learning to...,a: Predictive analytics | b: Natural language ...,a
2,Which field of study focuses on exploratory da...,a: Machine learning | b: Data mining | c: Deep...,b
3,What class of statistical algorithms have surp...,a: Neural networks | b: Decision trees | c: Su...,a
4,What provides a framework for describing machi...,a: Deep learning | b: Probably approximately c...,b


In [139]:
quiz=pd.DataFrame(quiz_table_data)

In [140]:
quiz.to_csv("ml.csv", index=False)